

# Project : Web Scraping offre d'emploi

In [1]:
import numpy as np
import os
import pandas as pd
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import urllib

from bs4 import BeautifulSoup
from time import sleep

### 1.1 Faire le scraping de liens avec les mots clés data scientist et data science: 
Source: https://www.mycareersfuture.sg

In [2]:
links = [
    "https://www.mycareersfuture.sg/search?search=data%20scientist&page={}",
    
 
]

job_postings = []

for link in links:
    for pages in range(0, 30 + 1):
        driver = webdriver.Edge(executable_path="C:\Windows\msedgedriver")

        driver.get(link.format(pages))

        sleep(5)

        html = BeautifulSoup(driver.page_source, "lxml")

        for job_card in html.find_all(
            "div", id=lambda x: x and x.startswith("job-card-")
        ):
            job_postings.append(
                "https://www.mycareersfuture.sg" + job_card.find("a")["href"]
            )

        driver.close()

In [7]:
#Removing duplicate links
job_postings = list(dict.fromkeys(job_postings))

In [8]:
len(job_postings)
# Close to requirement for at least 1,000 job listings. Moving to next stage

226

### 1.2 Création de fichier CSV avec job listings

In [9]:
company = []
title = []
salary = []
responsibilities = []
requirements = []
seniority = []
employment_type = []
category = []
address = []
failed_url = []

driver = webdriver.Edge(executable_path="C:\Windows\msedgedriver")

for index, link in enumerate(job_postings):

    try:
        driver.get(link)
        sleep(3)
        job_page = driver.page_source

        soup = BeautifulSoup(job_page, "lxml")

        company.append(soup.find("p", {"name": "company"}))
        title.append(soup.find("h1", {"id": "job_title"}))
        salary.append(soup.find("div", {"class": "lh-solid"}))
        responsibilities.append(soup.find("div", {"id": "job_description"}))
        requirements.append(soup.find("div", {"id": "requirements"}))
        seniority.append(soup.find("p", {"id": "seniority"}))
        employment_type.append(soup.find("p", {"id": "employment_type"}))
        category.append(soup.find("p", {"id": "job-categories"}))
        address.append(soup.find("p", {"id": "address"}))

    except:
        failed_url.append(link)

In [11]:
raw = [company, title, salary, responsibilities, requirements, seniority, employment_type, category, address]

In [12]:
clean = {}

for index, items in enumerate(raw):
    clean[index] = []
    for elements in items:
        try:
            clean[index].append(elements.text)
        except:
            clean[index].append(None)

In [13]:
data_jobs = pd.DataFrame(clean)

In [16]:
data_jobs.columns = [
    "Company",
    "Title",
    "Salary_Range",
    "Responsibilities",
    "Requirements",
    "Seniority",
    "Employment_Type",
    "Category",
    "Address",
]

In [17]:
data_jobs.head()

,Company,Title,Salary_Range,Responsibilities,Requirements,Seniority,Employment_Type,Category,Address
0,None,Data Scientist,$1 400to$1 400,Roles & ResponsibilitiesResponsibilities\n● De...,None,Fresh/entry level,Internship/Attachment,"Banking and Finance, Information Technology",410 NORTH BRIDGE ROAD 188726
1,None,Senior Data Scientist / Data Scientist,$9 500to$12 500,Roles & ResponsibilitiesAre you passionate abo...,None,Professional,"Permanent, Full Time",Engineering,"AIRLINE HOUSE, 25 AIRLINE ROAD 819829"
2,None,"Lead Data Scientist, Risk Data Mining, BRIC",$14 000to$28 000,Roles & ResponsibilitiesTikTok is the leading ...,None,Professional,Full Time,Information Technology,1 RAFFLES QUAY 048583
3,None,Data Scientist,$7 500to$9 500,Roles & ResponsibilitiesAre you passionate abo...,None,Professional,"Permanent, Full Time",Engineering,"AIRLINE HOUSE, 25 AIRLINE ROAD 819829"
4,None,Senior Data Scientist,$8 000to$16 000,Roles & ResponsibilitiesPrincipal Responsibili...,None,Professional,Full Time,Banking and Finance,"20 Pasir Panjang Rd., Mapletree Bus. City 117439"


In [23]:
data_jobs = data_jobs.to_csv('jobs.csv')

### DATA CLEANING AND EDA TO BE CONTINUED IN 2.0-cch-project4-EDA